Pulizia actors.csv e caricamento in PostGres

In [71]:
import pandas as pd
import os
import re
from unidecode import unidecode

In [72]:
# Ottieni la directory di lavoro corrente
script_dir = os.getcwd()

# Costruisci il percorso relativo al file CSV
file_path = os.path.join(script_dir, "Data", "main_data", "actors.csv")

# Costruisci il percorso relativo al file CSV pulito
clean_path = os.path.join(script_dir, "Data", "clean_data")

# Crea la cartella clean_data se non esite
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

# Caricare il CSV in un DataFrame
df = pd.read_csv(file_path)


In [73]:
# Rimuove duplicati
df.drop_duplicates(inplace=True)

In [74]:
# Rimuove righe con troppi valori nulli (ad esempio, più del 50% dei dati mancanti)
df.dropna(thresh=df.shape[1] * 0.5, inplace=True)

# Sostituisci i valori nulli con "Unknown" o il valore più comune
df['role'] = df['role'].fillna("Unknown")
df['name'] = df['name'].fillna("Unknown")

In [75]:
# Prende solo la prima parte prima di "/"
df['role'] = df['role'].str.split('/').str[0]

# Rimuove sinonimi tra parentesi
df['role'] = df['role'].apply(lambda x: re.sub(r"\(.*?\)|（.*?）", "", x))

# Standardizza il testo (se ci sono nomi di attori, ruoli, ecc.)
df['name'] = df['name'].str.strip().str.title()
df['role'] = df['role'].str.strip().str.lower()

print(df['role'].unique())

['barbie' 'ken' 'gloria' ... 'an du' 'ba cai' 'zuo yila']


In [76]:
# Funzione per tradurre il nome in caratteri latini 
def transliterate_name(name):
    if isinstance(name, str):  # Controlla che sia una stringa
        return unidecode(name).strip()  # Traduci e rimuovi spazi extra
    return name  # Se non è stringa, lascialo così

# Applica la funzione alla colonna dei nomi e dei ruoli
df['name'] = df['name'].apply(transliterate_name)
df['role'] = df['role'].apply(transliterate_name)


In [77]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "actors_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")